In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import dlib
import re

In [3]:
import cv2

# Define a few variables

In [4]:
basedir = './dataset_AMLS_22-23/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Define a set of functions to be used to extract the features 

In [5]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [6]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [7]:
def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

In [15]:
def resize_image(img_array):
    pil_image = Image.fromarray(img_array)
    resized_image = pil_image.resize((5, 5))
    return np.array(resized_image)

In [16]:
# This function will return the RBG values of the cropped images (we only care about the eye, not the whole face)
def get_eye_2(image_file):
    image = Image.open(f'dataset_AMLS_22-23/cartoon_set/img/{image_file}')
    image_array = np.array(image)
    features,_ = run_dlib_shape(image_array)
    eye_features = [features[37], features[38], features[40], features[41]]
    horizontal_pixels = [element[0] for element in eye_features]
    vertical_pixels = [element[1] for element in eye_features] 
    iris_crop = [(min(horizontal_pixels), max(horizontal_pixels)), (min(vertical_pixels), max(vertical_pixels))]
    image_array = image_array[iris_crop[1][0]:iris_crop[1][1], iris_crop[0][0]:iris_crop[0][1], :]
    resized_image_arr = resize_image(image_array)
    """
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    right_eye = img_array[250:277, 190:227, :]
    print(f'Completed eye {iteration}')
    return right_eye.reshape((1, 2997))
    """
    return resized_image_arr[:, :, 0:3].reshape((1, 75))/255